In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.gridspec import GridSpec
from scipy.special import boxcox1p
import warnings

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

import pandas as pd 
import numpy as np 
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold


from sklearn.compose import make_column_transformer
from itertools import combinations 


warnings.filterwarnings("ignore") # ignoring annoying warnings

In [2]:
df = pd.read_csv("titanic_total.csv")
target = "Survived"

In [3]:
df.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Deck,countDeck,Title,port_sex,Age_bin,FamilySize,Ticket_freq
0,1,0,3,male,1,0,7,S,U,1014,Mr,623,2,Small,1
1,2,1,1,female,1,0,71,C,C,94,Mrs,113,7,Small,2
2,3,1,3,female,0,0,7,S,U,1014,Miss,293,4,Alone,1
3,4,1,1,female,1,0,53,S,C,94,Mrs,293,6,Small,2
4,5,0,3,male,0,0,8,S,U,1014,Mr,623,6,Alone,1


In [4]:
fareGroups = ["low","medium","high","veryHigh", "superhigh"]
fareRanges = [-0.001, 7,10, 21,41, 512]
df["Fare_bin"] = pd.cut(df["Fare"], fareRanges, labels = fareGroups)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Deck,countDeck,Title,port_sex,Age_bin,FamilySize,Ticket_freq,Fare_bin
0,1,0,3,male,1,0,7,S,U,1014,Mr,623,2,Small,1,low
1,2,1,1,female,1,0,71,C,C,94,Mrs,113,7,Small,2,superhigh
2,3,1,3,female,0,0,7,S,U,1014,Miss,293,4,Alone,1,low
3,4,1,1,female,1,0,53,S,C,94,Mrs,293,6,Small,2,superhigh
4,5,0,3,male,0,0,8,S,U,1014,Mr,623,6,Alone,1,medium


In [6]:
df.drop("PassengerId", axis = 1, inplace = True)

In [9]:
df.to_csv("titanic_total_2.csv", index = False)

In [8]:
df.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Deck,countDeck,Title,port_sex,Age_bin,FamilySize,Ticket_freq,Fare_bin
0,0,3,male,1,0,7,S,U,1014,Mr,623,2,Small,1,low
1,1,1,female,1,0,71,C,C,94,Mrs,113,7,Small,2,superhigh
2,1,3,female,0,0,7,S,U,1014,Miss,293,4,Alone,1,low
3,1,1,female,1,0,53,S,C,94,Mrs,293,6,Small,2,superhigh
4,0,3,male,0,0,8,S,U,1014,Mr,623,6,Alone,1,medium


In [ ]:
####PROCESSING AND ANALYZING

In [ ]:
###ENCODING
#Selecciona todas las cols excepto target
df_sel = df.loc[:, df.columns != target]

cat_feats = df_sel.select_dtypes(include= object).columns
num_feats = df_sel.select_dtypes(exclude= object).columns


# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
enc_df = pd.DataFrame(enc.fit_transform(df_sel[cat_feats]).toarray())
print("Number of features onehotencoded:", enc_df.shape[1])
# Rejoin
df_sel = df_sel[num_feats].join(enc_df)

#Get back together with target
df = pd.concat((df[target], df_sel), axis = 1)

print("Final shape of all data after OneHotEncoding:", df.shape)

In [ ]:
df.head()

In [ ]:
# def single_value_dominate( df, target):
#         print("2.Removing columns with single value dominating > 95%:")
#         print("")
#         X2 = df.copy()
#         y2 = X2.pop(target)
#         remove_cols = []
#         for col in X2.columns:
#             count= 0
#             count = sum([+1 for i in X2[col].values if i == X2[col].mode()[0]])
#             if count/X2[col].shape[0] >= 0.95:
#                 remove_cols.append(col)
#         selected_cols = set(df.columns) - set(remove_cols)
#         print("Removed cols:", len(remove_cols))
#         print("Total selected cols:", len(selected_cols))
#         df = df[selected_cols]
#         return df


# df_svd =single_value_dominate(df, target = target )

In [ ]:
num_cols = df.select_dtypes(exclude= object).columns

In [ ]:
num_cols

In [ ]:
df[num_cols].std()

In [ ]:
num_cols = df.select_dtypes(include= np.number).columns
df[num_cols].dtypes
df2 = df[num_cols]
low = [col for col in df2.columns if df2[col].std() < 0.5]
low

In [ ]:
class Selector:
    def __init__(self):
        # self.target = target
        # self.df = df
        pass


    def variance_selector(self, df, target):
            #Rescale data if necessary
            print("1.Variance Threshold feature selection:")
            print("")
            X2 = df.copy()
            y2 = X2.pop(target)
            print("Initial features:", X2.shape[1])
            num_cols = X2.select_dtypes(include = np.number).columns
            low = [col for col in num_cols if X2[col].std() < 0.5]
            # print("Estos son antes de escale:", low)
            # scaler = StandardScaler().fit(X2)
            # XRescaled = scaler.transform(X2)
            # X_rescaled = pd.DataFrame(XRescaled, columns = X2.columns)
            # low = [col for col in X2.columns if X[col].std() < 0.5]
            #Analysis of amount of variation and droping all features with low variance
            var_tresh = VarianceThreshold(threshold = 0.5)
            var_tresh.fit_transform(X2[num_cols])
            data_transformed = X2[num_cols].loc[:, var_tresh.get_support()]
            #print("Selected feat:", data_transformed.columns)
            print("Removed features:", set(X2.columns) - set(data_transformed.columns))
            print("Final features:", data_transformed.shape[1])
            print("{} features with low variance removed".format(X2.shape[1] - data_transformed.shape[1]))
            #Rejoin
            df = pd.concat((y2, data_transformed), axis = 1)
            return df


    def single_value_dominate(self, df, target):
        print("2.Removing columns with single value dominating > 95%:")
        print("")
        X2 = df.copy()
        y2 = X2.pop(target)
        remove_cols = []
        for col in X2.columns:
            count= 0
            count = sum([+1 for i in X2[col].values if i == X2[col].mode()[0]])
            if count/X2[col].shape[0] >= 0.95:
                remove_cols.append(col)
        selected_cols = set(df.columns) - set(remove_cols)
        print("Removed cols:", len(remove_cols))
        print("Total selected cols:", len(selected_cols))
        df = df[selected_cols]
        return df


    def calcDrop(self, res):
        # All variables with correlation > cutoff
        all_corr_vars = list(set(res['v1'].tolist() + res['v2'].tolist()))

        # All unique variables in drop column
        poss_drop = list(set(res['drop'].tolist()))

        # Keep any variable not in drop column
        keep = list(set(all_corr_vars).difference(set(poss_drop)))

        # Drop any variables in same row as a keep variable
        p = res[ res['v1'].isin(keep)  | res['v2'].isin(keep) ][['v1', 'v2']]
        q = list(set(p['v1'].tolist() + p['v2'].tolist()))
        drop = (list(set(q).difference(set(keep))))

        # Remove drop variables from possible drop 
        poss_drop = list(set(poss_drop).difference(set(drop)))

        # subset res dataframe to include possible drop pairs
        m = res[ res['v1'].isin(poss_drop)  | res['v2'].isin(poss_drop) ][['v1', 'v2','drop']]

        # remove rows that are decided (drop), take set and add to drops
        more_drop = set(list(m[~m['v1'].isin(drop) & ~m['v2'].isin(drop)]['drop']))
        for item in more_drop:
            drop.append(item)

        return drop

    def corrX_new(self, df, target, cut) :
        print("")
        print("3. Removing features with high pairwise correlation")
        # Get correlation matrix and upper triagle
        df2 = df.drop(target, axis = 1)

        corr_mtx = df2.corr().abs()
        avg_corr = corr_mtx.mean(axis = 1)
        up = corr_mtx.where(np.triu(np.ones(corr_mtx.shape), k=1).astype(np.bool))

        dropcols = list()

        res = pd.DataFrame(columns=(['v1', 'v2', 'v1.target', 
                                        'v2.target','corr', 'drop' ]))

        for row in range(len(up)-1):
            col_idx = row + 1
            for col in range (col_idx, len(up)):
                if(corr_mtx.iloc[row, col] > cut):
                    if(avg_corr.iloc[row] > avg_corr.iloc[col]): 
                        dropcols.append(row)
                        drop = corr_mtx.columns[row]
                    else: 
                        dropcols.append(col)
                        drop = corr_mtx.columns[col]

                    s = pd.Series([ corr_mtx.index[row],
                    up.columns[col],
                    avg_corr[row],
                    avg_corr[col],
                    up.iloc[row,col],
                    drop],
                    index = res.columns)

                    res = res.append(s, ignore_index = True)

        dropcols_names = self.calcDrop(res)
        print("{} features removed".format(len(dropcols_names)))
        print("Features removed:", dropcols_names)
        print("")
        print("Selected features:", df.shape[1] - len(dropcols_names))
        selected_cols = set(df.columns) - set(dropcols_names)
        print("")
        print("Selected:", selected_cols)
        return df[selected_cols]
    
    def corr_target(self, df):
        print("")
        print("Remove features with low target correlation:")
        #Removes all feats with correlation under threshold
        remove_features = [feat for feat in df.columns if df.corr().abs()[[target]].loc[feat, :][0] < 0.05]
        selected_cols = set(df.columns) - set(remove_features)
        print("Total features removed:", len(remove_features))
        print("Nr of features selected:", len(selected_cols))
        print("Final features selected ", selected_cols)

        return df[selected_cols]

In [ ]:
slc = Selector()
# df_r = slc.rescale(df, target = "Survived")
df_v = slc.variance_selector(df, target = target)
df_svd =slc.single_value_dominate(df_v, target = target )
df_corr_f = slc.corrX_new(df_svd, target = target, cut = 0.65)
#Aqui multicollinearity
# print("4. Multicollinearity analysis")
# df_multi = slc.calculate_vif_(df_corr_f, thresh=5.0)
#Correlation with target
df_corr_target = slc.corr_target(df_corr_f)

#Update train and test sets with selected features
df_final = df[df_corr_target.columns]


In [ ]:
df_final.corr()

In [ ]:
df.corr()

In [ ]:
df.corr()["PassengerId"]